# Micro-Autograd — Rétropropagation depuis Zéro

Implémentation d'un moteur de différentiation automatique scalaire, inspiré de [micrograd](https://github.com/karpathy/micrograd) (A. Karpathy).

## Objectifs
- Implémenter la classe `Value` avec typage complet
- Supporter : `+`, `-`, `*`, `/`, `**`, `relu`, `exp`, `log`
- Vérifier les gradients par différences finies (gradient check)
- Simuler un neurone complet (forward + backward)

In [1]:
from __future__ import annotations
import math
from typing import Callable, Union

Number = Union[int, float]

## 1. La classe `Value`

Encapsule un scalaire et construit le graphe de calcul au fur et à mesure.
Chaque nœud stocke :
- `data` : valeur numérique
- `grad` : gradient accumulé (init. à `0.0`)
- `_prev` : nœuds parents (pour remonter le graphe)
- `_backward` : fermeture calculant le gradient local

In [2]:
class Value:
    """Scalaire avec différentiation automatique (reverse mode)."""

    def __init__(
        self,
        data: float,
        _children: tuple[Value, ...] = (),
        _op: str = "",
    ) -> None:
        self.data: float = float(data)
        self.grad: float = 0.0
        self._prev: set[Value] = set(_children)
        self._op: str = _op
        self._backward: Callable[[], None] = lambda: None

    def __repr__(self) -> str:
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"

    # ------------------------------------------------------------------
    # Opérations arithmétiques
    # ------------------------------------------------------------------

    def __add__(self, other: Value | Number) -> Value:
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")

        def _backward() -> None:
            self.grad  += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __radd__(self, other: Number) -> Value:          # other + self
        return self + other

    def __neg__(self) -> Value:
        return self * -1

    def __sub__(self, other: Value | Number) -> Value:
        return self + (-other)

    def __rsub__(self, other: Number) -> Value:          # other - self
        return Value(other) + (-self)

    def __mul__(self, other: Value | Number) -> Value:
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        def _backward() -> None:
            self.grad  += other.data * out.grad
            other.grad += self.data  * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other: Number) -> Value:          # other * self
        return self * other

    def __pow__(self, exponent: int | float) -> Value:
        assert isinstance(exponent, (int, float)), "exposant doit être un scalaire"
        out = Value(self.data ** exponent, (self,), f"**{exponent}")

        def _backward() -> None:
            self.grad += exponent * (self.data ** (exponent - 1)) * out.grad

        out._backward = _backward
        return out

    def __truediv__(self, other: Value | Number) -> Value:
        other_val = other if isinstance(other, Value) else Value(other)
        return self * other_val ** -1

    def __rtruediv__(self, other: Number) -> Value:      # other / self
        return Value(other) * self ** -1

    # ------------------------------------------------------------------
    # Fonctions d'activation et mathématiques
    # ------------------------------------------------------------------

    def relu(self) -> Value:
        out = Value(max(0.0, self.data), (self,), "ReLU")

        def _backward() -> None:
            self.grad += (out.data > 0) * out.grad

        out._backward = _backward
        return out

    def exp(self) -> Value:
        val = math.exp(self.data)
        out = Value(val, (self,), "exp")

        def _backward() -> None:
            self.grad += val * out.grad      # d/dx exp(x) = exp(x)

        out._backward = _backward
        return out

    def log(self) -> Value:
        assert self.data > 0, f"log non défini pour data={self.data:.4f}"
        out = Value(math.log(self.data), (self,), "log")

        def _backward() -> None:
            self.grad += (1.0 / self.data) * out.grad

        out._backward = _backward
        return out

    # ------------------------------------------------------------------
    # Backward pass
    # ------------------------------------------------------------------

    def backward(self) -> None:
        """Rétropropagation par tri topologique (reverse mode)."""
        topo: list[Value] = []
        visited: set[Value] = set()

        def build_topo(v: Value) -> None:
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

## 2. Forward Pass — Tests des opérations

In [3]:
x = Value(2.0)
y = Value(-3.0)

# Addition
z = x + y
assert z.data == -1.0
print(f"x + y   = {z}")

# Scalaire à droite
w = x + 3
assert w.data == 5.0
print(f"x + 3   = {w}")

# Multiplication
p = x * y
assert p.data == -6.0
print(f"x * y   = {p}")

# Négation
print(f"-x      = {-x}")

# Soustraction
s = x - y
assert s.data == 5.0
print(f"x - y   = {s}")

# Division
d = x / Value(-4.0)
assert abs(d.data - (-0.5)) < 1e-9
print(f"x / -4  = {d}")

# Puissance
pw = x ** 3
assert pw.data == 8.0
print(f"x**3    = {pw}")

x + y   = Value(data=-1.0000, grad=0.0000)
x + 3   = Value(data=5.0000, grad=0.0000)
x * y   = Value(data=-6.0000, grad=0.0000)
-x      = Value(data=-2.0000, grad=0.0000)
x - y   = Value(data=5.0000, grad=0.0000)
x / -4  = Value(data=-0.5000, grad=0.0000)
x**3    = Value(data=8.0000, grad=0.0000)


In [4]:
# ReLU
assert Value( 3.0).relu().data == 3.0
assert Value(-2.0).relu().data == 0.0
print(f"relu( 3.0) = {Value( 3.0).relu().data}")
print(f"relu(-2.0) = {Value(-2.0).relu().data}")

# Exponentielle
e_val = Value(1.0).exp()
assert abs(e_val.data - math.e) < 1e-10
print(f"exp(1.0)   = {e_val.data:.6f}")

# Logarithme
l_val = Value(math.e).log()
assert abs(l_val.data - 1.0) < 1e-10
print(f"log(e)     = {l_val.data:.6f}")

relu( 3.0) = 3.0
relu(-2.0) = 0.0
exp(1.0)   = 2.718282
log(e)     = 1.000000


## 3. Backward Pass — Calcul des gradients

Le backward pass utilise un **tri topologique** du graphe de calcul pour propager les gradients de la sortie vers les entrées.

**Règle générale :** `node.grad += gradient_amont × dérivée_locale`

In [5]:
# f(x, y) = x * y + x
# df/dx = y + 1 = -3 + 1 = -2
# df/dy = x = 2
x = Value(2.0)
y = Value(-3.0)
z = x * y + x
z.backward()

print(f"z = x*y + x       = {z.data}")
print(f"dz/dx = {x.grad:.1f}   (attendu : {float(-3 + 1):.1f})")
print(f"dz/dy = {y.grad:.1f}    (attendu : {2.0:.1f})")
assert abs(x.grad - (-2.0)) < 1e-9
assert abs(y.grad -   2.0)  < 1e-9

z = x*y + x       = -4.0
dz/dx = -2.0   (attendu : -2.0)
dz/dy = 2.0    (attendu : 2.0)


In [6]:
# f(x) = relu(x^2 - 4)  à x = 3
# x^2 = 9  =>  9 - 4 = 5  =>  relu(5) = 5
# df/dx = relu'(5) * 2x = 1 * 6 = 6
x = Value(3.0)
y = (x ** 2 - 4.0).relu()
y.backward()

print(f"y = relu(x²-4) = {y.data}")
print(f"dy/dx = {x.grad:.1f}   (attendu : {2 * 3.0:.1f})")
assert abs(x.grad - 6.0) < 1e-9

y = relu(x²-4) = 5.0
dy/dx = 6.0   (attendu : 6.0)


## 4. Vérification Numérique des Gradients (Gradient Check)

La **différence finie centrée** estime le gradient numériquement :

$$\frac{\partial f}{\partial x_i} \approx \frac{f(\ldots,\, x_i + \varepsilon,\, \ldots) - f(\ldots,\, x_i - \varepsilon,\, \ldots)}{2\varepsilon}$$

On compare cette estimation au gradient calculé par autograd.

In [7]:
def grad_check(
    f: Callable[..., Value],
    inputs: list[float],
    eps: float = 1e-5,
    tol: float = 1e-4,
) -> None:
    """Vérifie les gradients autograd par différences finies centrées."""
    vals = [Value(x) for x in inputs]
    out = f(*vals)
    out.backward()
    ag = [v.grad for v in vals]

    print(f"  {'var':<6} {'autograd':>12} {'numérique':>12} {'ok':>4}")
    print(f"  {'-' * 38}")
    all_ok = True
    for i, (a, inp) in enumerate(zip(ag, inputs)):
        inp_p = inputs.copy(); inp_p[i] += eps
        inp_m = inputs.copy(); inp_m[i] -= eps
        ng = (
            f(*[Value(x) for x in inp_p]).data
          - f(*[Value(x) for x in inp_m]).data
        ) / (2 * eps)
        ok = abs(a - ng) < tol
        all_ok = all_ok and ok
        status = "✓" if ok else "✗"
        print(f"  x_{i:<4} {a:>12.6f} {ng:>12.6f} {status:>4}")
    print()
    assert all_ok, "Gradient check échoué !"

In [8]:
print("f(x, y) = x * y")
grad_check(lambda x, y: x * y, [2.0, -3.0])

print("f(x) = x ** 3")
grad_check(lambda x: x ** 3, [2.0])

print("f(x) = exp(x)")
grad_check(lambda x: x.exp(), [1.0])

print("f(x) = log(x)")
grad_check(lambda x: x.log(), [2.0])

print("f(x) = relu(x)  [x > 0]")
grad_check(lambda x: x.relu(), [1.5])

print("f(x) = relu(x)  [x < 0]")
grad_check(lambda x: x.relu(), [-1.5])

print("f(x, y) = relu(x * y + x)")
grad_check(lambda x, y: (x * y + x).relu(), [1.0, 1.0])

print("Tous les gradients sont corrects ✓")

f(x, y) = x * y
  var        autograd    numérique   ok
  --------------------------------------
  x_0       -3.000000    -3.000000    ✓
  x_1        2.000000     2.000000    ✓

f(x) = x ** 3
  var        autograd    numérique   ok
  --------------------------------------
  x_0       12.000000    12.000000    ✓

f(x) = exp(x)
  var        autograd    numérique   ok
  --------------------------------------
  x_0        2.718282     2.718282    ✓

f(x) = log(x)
  var        autograd    numérique   ok
  --------------------------------------
  x_0        0.500000     0.500000    ✓

f(x) = relu(x)  [x > 0]
  var        autograd    numérique   ok
  --------------------------------------
  x_0        1.000000     1.000000    ✓

f(x) = relu(x)  [x < 0]
  var        autograd    numérique   ok
  --------------------------------------
  x_0        0.000000     0.000000    ✓

f(x, y) = relu(x * y + x)
  var        autograd    numérique   ok
  --------------------------------------
  x_0        2.

## 5. Exemple Complet — Un Neurone

On simule un neurone ReLU à deux entrées :

$$\hat{y} = \text{ReLU}(w_1 x_1 + w_2 x_2 + b)$$

et on calcule le gradient de la sortie par rapport à chaque paramètre.

In [9]:
# Entrées
x1 = Value(2.0)
x2 = Value(3.0)

# Paramètres (à apprendre par gradient descent)
w1 = Value( 0.5)
w2 = Value(-0.5)
b  = Value( 1.0)

# ── Forward ───────────────────────────────────────────────────────────
pre_act = w1 * x1 + w2 * x2 + b   # 0.5*2 + (-0.5)*3 + 1.0 = 0.5
out     = pre_act.relu()           # ReLU(0.5) = 0.5

print("── Forward Pass ───────────────────────────────")
print(f"  w1*x1 + w2*x2 + b  = {pre_act.data:.4f}")
print(f"  ReLU(pré-activation) = {out.data:.4f}")

# ── Backward ──────────────────────────────────────────────────────────
out.backward()

print()
print("── Gradients ∂out/∂· ──────────────────────────")
print(f"  ∂/∂w1 = {w1.grad:.4f}   (= x1 = {x1.data})")
print(f"  ∂/∂w2 = {w2.grad:.4f}   (= x2 = {x2.data})")
print(f"  ∂/∂b  = {b.grad:.4f}    (= 1)")
print(f"  ∂/∂x1 = {x1.grad:.4f}   (= w1 = {w1.data})")
print(f"  ∂/∂x2 = {x2.grad:.4f}  (= w2 = {w2.data})")

── Forward Pass ───────────────────────────────
  w1*x1 + w2*x2 + b  = 0.5000
  ReLU(pré-activation) = 0.5000

── Gradients ∂out/∂· ──────────────────────────
  ∂/∂w1 = 2.0000   (= x1 = 2.0)
  ∂/∂w2 = 3.0000   (= x2 = 3.0)
  ∂/∂b  = 1.0000    (= 1)
  ∂/∂x1 = 0.5000   (= w1 = 0.5)
  ∂/∂x2 = -0.5000  (= w2 = -0.5)
